In [ ]:
import os
os.environ["PATH"] = '/home/mattjones/bin:/home/mattjones/.local/bin:/home/mattjones/myapps/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/mattjones/opt/gurobi801/linux64/bin:/home/mattjones/software/bowtie2-2.3.4.2:/home/mattjones/emboss/EMBOSS-6.6.0/emboss'
os.environ["LD_LIBRARY_PATH"] = '/home/mattjones/lib/::/home/mattjones/opt/gurobi801/linux64/lib'

from Cassiopeia.TreeSolver import data_pipeline
from Cassiopeia.TreeSolver.simulation_tools import dataset_generation
from Cassiopeia.TreeSolver import score_triplets

import pandas as pd 
import numpy as np
import networkx as nx
import pickle as pic

import sys

### Simulate Tree

To simulate the tree, we will use the function ``generate_simulated_full_tree``. This function will begin with a single cell and every generation create two offspring (i.e. at the end we will be left with $2^D$ cells). Every time a cell divides, we first create mutations -- this process entails considering every character and mutating to a state, specified by the mutation map, if we draw a probability less than the mutation probability specified. At the end of the experiment, we subsample ``sub_samp`` percent of the cells from the total $2^D$ cells. The parameters we use here are:

- **D**: The depth of the tree, or number of generations to simulate 
- **C**: The number of characters to simulate
- **S**: The number of states per character allowed.  
- **sub_samp**: The proportion of cells to sample at the end of the simulatoin
- **M**: The per character mutation rate (i.e. Cas9-cutting rate). 
- **mutation_map**: This will be a nested dictionary for every character, storing the relative propensities to mutate to a given state. Many models can be used, here we use a simple negative binomial model. 

This function will return a Cassiopeia_Tree object that stores the simulated network.

In [ ]:
# set parameters
D = 12
C = 40
S = 20
num_cells = 400
sub_samp = num_cells / 2**D
M = 0.0275

dropout = dict(zip(range(C), np.random.randint(7, high=40, size=C) / 100))

# Create mutation map.  
mutation_map = {}
for i in range(C):
    sampled_probabilities = sorted([np.random.negative_binomial(5, .5) for _ in range(1, S)])
    mutation_map[i] = {'0': (1 - M)}
    total = np.sum(sampled_probabilities)
    for j in range(1, S):
        mutation_map[i][str(j)] = (M)*sampled_probabilities[j-1]/(1.0 * total)

In [ ]:
true_network = dataset_generation.generate_simulated_full_tree(mutation_map, dropout, characters=C, subsample_percentage = sub_samp, depth=D)

pic.dump(true_network, open("true_network_test.pkl", "wb"))

### Reconstruct Trees with the Stress-Test Module

You can use the `stress-test` command line tool to reconstruct the trees from the simulation. This module will take in the Cassiopeia_Tree pickle file produced from the `generate_simulated_full_tree` function and reconstruct a tree using any of five algorithms. Use the `-h` flag to see usage and all options.

Below, we have used already run the stress test command on the true network and have saved the result in `greedy_network_test.pkl` (this will be a Cassiopeia_Tree as well, storing the reconstruction)

### Validation

Validation consists of using our *triplets correct* statistic, which can be called in the ``score_triplets`` function. This will take in the true network & the reconstructed network and compare several triplets between the two trees, both of which should be Cassiopeia_Trees. 

This function takes other hyperparameters:

- **modified**: weight by depth
- **number_of_trials**: number of triplets to sample & compare
- **min_size_depth**: minimum number of observations at a given depth required for triplets at that depth to be considered.

The function will return a float between 0 and 1 representing the similarity between the trees. 

In [ ]:
reconstructed_tree = nx.read_gpickle("greedy_network_test.pkl")
true_network = nx.read_gpickle("true_network_test.pkl")

triplet_score = score_triplets.score_triplets(true_network, reconstructed_tree, modified=True, number_of_trials=10000, min_size_depth = 20)

print("Triplet score: " + str(triplet_score))